In [165]:
#imports
import pandas as pd 
from pandas import DataFrame
from slugify import slugify
import requests
from tqdm.notebook import tqdm
from time import sleep
from bs4 import BeautifulSoup as bs
import json
from datetime import datetime
import webbrowser
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
from time import sleep

In [166]:
#getting both csv files 
movie_details = pd.read_csv('/Users/armaan/src/bbox/mdata/thenum/movie_details.csv') 
imdb_movies = pd.read_csv('/Users/armaan/src/bbox/mdata/imdb/imdb_movies.csv') 
#converting names to lower and some cols to int
movie_details['name_lower'] = movie_details['name'].str.lower()
imdb_movies['year'] = pd.to_numeric(imdb_movies.startYear, errors='coerce')
imdb_movies['primaryTitle_lower'] = imdb_movies['primaryTitle'].str.lower()
imdb_movies['runtimeMinutes'] = pd.to_numeric(imdb_movies.runtimeMinutes, errors='coerce')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [167]:
#normalization
def normalize(x):
    if x:
        #for star wars
        x = x.replace('-ep-', '-episode-')
        #for harry potter
        x = x.replace('part-ii', 'part-2')
        x = x.replace('part-i', 'part-1')
        #for e.t. only!
        if x.startswith('et-'):
            x = x.replace('et-', 'e-t-')
        #for rise of skywalker only!
        if x == 'star-wars-the-rise-of-skywalker':
            x = "star-wars-episode-ix-the-rise-of-skywalker"
    return x

#find duplicates
def duplicate_scraper(imdb_id):
    r = requests.get("https://www.imdb.com/title/{}/".format(imdb_id))
    soup = bs(r.text)
    box_office = soup.find("div",{"id": "titleDetails"})
    all_h4 = box_office.findAll("h4")
    drop = True
    for h4 in all_h4:
        if h4.string == "Gross USA:":
            drop = False
            break
    return drop

In [168]:
#sluggifying and normalizing all names
movie_details['name_slug'] = movie_details.name_lower.apply(slugify)
imdb_movies['name_slug'] = imdb_movies.primaryTitle_lower.apply(slugify)
movie_details['name_slug'] = movie_details.name_slug.apply(normalize)
#creating new df with updated values
new_df = pd.merge(movie_details, imdb_movies, how ='inner', right_on = ['name_slug', 'year'], left_on = ['name_slug', 'year'])

In [169]:
#find duplicates and get rid of them
duplicates = new_df[new_df.name.duplicated()]
for index, row in tqdm(duplicates.iterrows()):
    match = new_df[(new_df['name'] == row["name"]) & (new_df['year'] == row["year"])]
    if len(match) > 1:
        imdb_ids = match["tconst"]
        for imdb_id in imdb_ids:
            drop = duplicate_scraper(imdb_id)
            if drop == True:
                new_df.drop( new_df[ new_df['tconst'] == imdb_id ].index , inplace=True)

In [170]:
#drops
new_df[new_df.tconst != 'tt11861230']
new_df.drop(["endYear", "isAdult", "titleType", "startYear", "primaryTitle_lower", "runtimeMinutes", "genres", "name_lower", "name_slug", "primaryTitle", "originalTitle"], axis=1, inplace = True)

In [190]:
new_df[:260]

,name,year,domestic_box_office,international_box_office,worldwide_box_office,opening_weekend,legs,production_budget,theater_counts,mpaa_rating,...,genre,production_method,creative_type,production_companies,synopsis,link,tconst,poster_path,similar_movies,imdb_score
0,Star Wars Ep. VII: The Force Awakens,2015,936662225,1131561399.00,2068223624.00,247966675.00,3.78,306000000.00,9.20,PG-13,...,Adventure,Animation/Live Action,Science Fiction,Lucasfilm,"Rey, a scavenger from the planet Jakku, and re...",https://www.the-numbers.com/movie/Star-Wars-Ep...,tt2488496,https://image.tmdb.org/t/p/w300_and_h450_bestv...,"[{'name': 'Star Trek Into Darkness', 'link': '...",7.9
1,Avengers: Endgame,2019,858373000,1939427564.00,2797800564.00,357115007.00,3.78,400000000.00,8.10,PG-13,...,Action,Animation/Live Action,Super Hero,Marvel Studios,The grave course of events set in motion by Th...,https://www.the-numbers.com/movie/Avengers-End...,tt4154796,https://image.tmdb.org/t/p/w300_and_h450_bestv...,"[{'name': 'Captain Marvel', 'link': 'Captain M...",8.4
2,Avatar,2009,760507625,2028193712.00,2788701337.00,77025481.00,3.78,237000000.00,13.20,PG-13,...,Action,Animation/Live Action,Science Fiction,Dune Entertainment,"Jake Sully is a wounded ex-marine, thrust into...",https://www.the-numbers.com/movie/Avatar#tab=s...,tt0499549,https://image.tmdb.org/t/p/w300_and_h450_bestv...,"[{'name': 'Interstellar', 'link': 'Interstella...",7.8
3,Black Panther,2018,700059566,646853595.00,1346913161.00,202003951.00,3.78,200000000.00,9.90,PG-13,...,Action,Live Action,Super Hero,Marvel Studios,"King T'Challa returns home to the reclusive, t...",https://www.the-numbers.com/movie/Black-Panthe...,tt1825683,https://image.tmdb.org/t/p/w300_and_h450_bestv...,"[{'name': 'X-Men: Apocalypse', 'link': 'X-Men:...",7.3
4,Avengers: Infinity War,2018,678815482,1369544272.00,2048359754.00,257698183.00,3.78,300000000.00,7.70,PG-13,...,Action,Animation/Live Action,Super Hero,Marvel Studios,As the Avengers and their allies have continue...,https://www.the-numbers.com/movie/Avengers-Inf...,tt4154756,https://image.tmdb.org/t/p/w300_and_h450_bestv...,"[{'name': 'Captain Marvel', 'link': 'Captain M...",8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,Captain America: The First Avenger,2011,176654505,193915271.00,370569776.00,65058524.00,3.78,140000000.00,5.70,PG-13,...,Action,Live Action,Super Hero,Paramount Pictures,Captain America: The First Avenger will focus ...,https://www.the-numbers.com/movie/Captain-Amer...,tt0458339,https://image.tmdb.org/t/p/w300_and_h450_bestv...,"[{'name': 'The Incredible Hulk', 'link': 'The ...",6.9
267,The Karate Kid,2010,176591618,175183320.00,351774938.00,55665805.00,3.78,40000000.00,5.70,PG,...,Adventure,Live Action,Contemporary Fiction,Columbia Pictures,NaN,https://www.the-numbers.com/movie/Karate-Kid-T...,tt1155076,https://image.tmdb.org/t/p/w300_and_h450_bestv...,[],6.2
268,There's Something About Mary,1998,176484651,193400000.00,369884651.00,13740644.00,3.78,22000000.00,15.50,R,...,Romantic,Live Action,Contemporary Fiction,20th Century Fox,NaN,https://www.the-numbers.com/movie/Theres-Somet...,tt0129387,https://image.tmdb.org/t/p/w300_and_h450_bestv...,"[{'name': 'The Ugly Truth', 'link': 'The Ugly ...",7.1
269,Ice Age,2002,176387405,209728938.00,386116343.00,46312454.00,3.78,65000000.00,9.00,PG,...,Adventure,Digital Animation,Kids Fiction,Blue Sky Studios,NaN,https://www.the-numbers.com/movie/Ice-Age#tab=...,tt0268380,https://image.tmdb.org/t/p/w300_and_h450_bestv...,"[{'name': 'The Croods', 'link': 'The Croods^20...",7.5


In [172]:
#get poster path and add it to new col in df
poster_link_list = []
for p in tqdm(new_df['tconst']):
    tmdb = requests.get("https://api.themoviedb.org/3/movie/{}?api_key=a6837e56522d89b354d01b07e393bebc&language=en-US".format(p))
    try:
        poster_path = tmdb.json()['poster_path']
        poster_link = "https://image.tmdb.org/t/p/w300_and_h450_bestv2/{}".format(poster_path)
    except:
        if p:
            print ("no poster for " + p)
    poster_link_list.append(poster_link)

new_df["poster_path"] = poster_link_list

In [173]:
#drop movies with missing information
new_df
new_df2 = new_df.dropna(subset=['domestic_box_office', 'international_box_office', 'worldwide_box_office', 'production_budget', 'opening_weekend'])
new_df = new_df2

In [174]:
#get similar movies
def get_matching_similar(df_movies, simlist, how_many=5):
    matching_movies = []
    result_cnt = min(len(simlist), how_many)
    for s in simlist:
        try:
            title = s['title']
            release = int(s['release_date'][:4])
            if len(df_movies[(df_movies.name == title) & (df_movies.year == release)]) == 1:
                poster_path = new_df.loc[new_df['name'] == title, 'poster_path'].values[0]
                name = new_df.loc[new_df['name'] == title, 'name'].values[0]
                movie_link = "{}^{}".format(title, release)
                matching_movies.append({'name': name, 'link': movie_link, 'poster' : poster_path})
                
                if len(matching_movies) >= how_many:
                    break
        except Exception as ex:
            print(ex)

            
    return matching_movies


similar_movies_list = []
x = 0
for t in tqdm(new_df['tconst']):
    turl = "https://api.themoviedb.org/3/movie/{}/similar?api_key=a6837e56522d89b354d01b07e393bebc&language=en-US&page=1".format(t)
    tmdb = requests.get(turl)
    if 'results' in tmdb.json():
        sim_list = tmdb.json()['results']
    else:
        sim_list = []
    similar_movies_list.append(get_matching_similar(new_df, sim_list, how_many=5))
#     x = x+1
#     if x > 3:
#         break
        
new_df["similar_movies"] = similar_movies_list

'release_date'
'release_date'
'release_date'
'release_date'
'release_date'
'release_date'
invalid literal for int() with base 10: ''
'release_date'
invalid literal for int() with base 10: ''



/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [175]:
#scrape for imdb score
imdb_scores = []
x = 0
for t in tqdm(new_df['tconst']):
    r = requests.get("https://www.imdb.com/title/{}/".format(t))
    soup = bs(r.text)
    score_section = soup.find("span", {"itemprop": "ratingValue"})
    score = score_section.string
    imdb_scores.append(score)
#     x = x+1
#     if x > 3:
#         break
     
new_df["imdb_score"] = imdb_scores

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [191]:
bbox_df = new_df2

bbox_df['bb_profit'] = bbox_df.worldwide_box_office - bbox_df.production_budget
bbox_df['bb_profit_multiple'] = bbox_df.worldwide_box_office/bbox_df.worldwide_box_office

bbox_df['bb_profit_score'] = (bbox_df.bb_profit / bbox_df.bb_profit.max()) * 100
bbox_df['bb_profit_multiple_score'] = (bbox_df.bb_profit_multiple / bbox_df.bb_profit_multiple.max()) * 100

bbox_df['bb_score'] = (bbox_df['bb_profit_score']*0.7 + bbox_df['bb_profit_multiple_score']*0.3)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [193]:
#downloads df to csv
bbox_df = bbox_df[~bbox_df.worldwide_box_office.isnull()]
bbox_df.to_csv('/Users/armaan/src/bbox/mdata/final_result/master_table.csv', index=False)